In [3]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np


def read_jsonl(filename):
    """Read data from a jsonl file."""
    data = []
    with open(filename, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data


def draw_graph(data, output_filename):
    """Draw a graph with improved positioning for isolated nodes."""
    G = nx.Graph()

    # Add nodes and edges to the graph
    for node in data["nodes"]:
        G.add_node(node["id"])
    for link in data["links"]:
        G.add_edge(link["source"], link["target"], weight=link["weight"])

    # Identify isolated nodes
    isolated_nodes = list(nx.isolates(G))
    connected_nodes = list(set(G.nodes()) - set(isolated_nodes))

    # Use spring layout for the main graph and then adjust for isolated nodes
    pos = nx.spring_layout(G)

    # If there are isolated nodes, position them in a circle around the graph
    if isolated_nodes:
        border_positions = {}
        border_angle = 2 * 3.141592653589793 / len(isolated_nodes)
        radius = 1.2  # To place isolated nodes outside the main cluster
        for index, node in enumerate(isolated_nodes):
            angle = index * border_angle
            border_positions[node] = (radius * np.cos(angle), radius * np.sin(angle))
        pos.update(border_positions)

    # Draw the graph
    plt.figure(figsize=(10, 10))
    nx.draw_networkx_nodes(G, pos, nodelist=connected_nodes, node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=isolated_nodes, node_size=500, node_color='lightblue', node_shape='s')
    nx.draw_networkx_labels(G, pos, font_size=10)
    edge_widths = [d["weight"] / 10 for _, _, d in G.edges(data=True)]
    edge_labels = {(u, v): round(d["weight"], 1) for u, v, d in G.edges(data=True)}
    nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.6)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=6)

    plt.axis("off")
    plt.tight_layout()
    plt.savefig(output_filename, format="PNG", dpi=300)
    plt.close()


def main(input_filename="gh_output_graphs.jsonl"):
    data_list = read_jsonl(input_filename)
    for idx, data in enumerate(data_list):
        draw_graph(data, f"assets/gh_user_graph_{idx}.png")


if __name__ == "__main__":
    main()